<a href="https://colab.research.google.com/github/YousefAli176/YousefAli176/blob/main/Copy_of_fine_tuning_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install transformers
import transformers
from transformers import GPT2Tokenizer,GPT2LMHeadModel

In [ ]:
#!pip install datasets
#!pip install accelerate -U
#run one at each time and then restart runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
model_name="gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
input_data=" python can be very"
max_length=100
num_return_sequences=1
input_ids = tokenizer(input_data,return_tensors="pt")["input_ids"]


In [ ]:
output = model.generate(input_ids,
                        max_length=max_length,
                        num_return_sequences=num_return_sequences,  # Fixed parameter
                        temperature=0.7,
                        top_k=20,  # for more diverse sampling
                        no_repeat_ngram_size=2
                        )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
#model.generate??

In [ ]:
for i, sequence in enumerate(output):
    print(f"Generated sequence before fine tuning  {i + 1}: {tokenizer.decode(sequence)}")
    print("\nEnd of sequence \n")

Generated sequence before fine tuning  1:  python can be very useful for debugging.

The following code snippet shows how to use the debugger to debug a program. It is a simple example of how the program can run in a debugger. The code is written in C++. You can see the code in the source code.<|endoftext|>

End of sequence 



In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [ ]:
from transformers import TrainingArguments, TextDataset, DataCollatorForLanguageModeling, Trainer

from datasets import Dataset

In [ ]:
#dataset = load_dataset('bookcorpus')

In [ ]:
#dataset

In [ ]:
#travel_keywords = r"(?i)\b(Python)\b"

In [ ]:
#import re

#filtered_dataset = dataset["train"].filter(
 #   lambda x: re.compile(travel_keywords).search(x["text"]) is not None
#)


In [ ]:
#filtered_dataset

In [ ]:
#filtered_dataset.to_csv('travel.csv', index=False)

In [ ]:
import pandas as pd

# Assuming your dataset is named "my_dataset.csv"
travel_data = pd.read_csv("/content/sample_data/python.csv")
travel_data =travel_data[:1000]

In [ ]:
import re
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text)  # Remove extra spaces
    return text.lower()  # Convert to lowercase

travel_data['text'] = travel_data['text'].apply(clean_text)


In [ ]:
# Clean travel descriptions
travel_data["cleaned_text"] = travel_data["text"].apply(clean_text)

# Create Hugging Face Dataset
train_dataset = Dataset.from_pandas(travel_data[["cleaned_text"]])



In [ ]:
print(len(train_dataset))

736


In [ ]:
train_dataset[10]

{'cleaned_text': 'an image popped into his head glowing just like the grailshaped beacon in that monty python moviea bottle of bourbon '}

In [ ]:
#with open("data.txt", "w") as f:
  #  for data in train_dataset["cleaned_text"]:
    #    f.write(" ".join(map(str, data)) + "\n")

In [ ]:
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/sample_data/python.csv",
    block_size=128,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
#!pip install accelerate -U
#!pip install accelerate
#!pip install transformers[torch]

In [ ]:
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,  # Adjust number of epochs
    per_device_train_batch_size=4,  # Adjust batch size based on GPU memory
    save_steps=200,
    save_total_limit=2,  # Save only the two best checkpoints
)

In [ ]:
def load_data_collator(tokenizer, mlm = False):
    return DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
data_collator1=load_data_collator(tokenizer,False)

In [ ]:
# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator1,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=180, training_loss=3.911082288953993, metrics={'train_runtime': 1743.4828, 'train_samples_per_second': 0.408, 'train_steps_per_second': 0.103, 'total_flos': 46444658688000.0, 'train_loss': 3.911082288953993, 'epoch': 3.0})

In [ ]:
trainer.save_model()
model2 = GPT2LMHeadModel.from_pretrained("/content/output")

In [ ]:
model2 = GPT2LMHeadModel.from_pretrained("/content/output")

In [ ]:
#model2??

In [ ]:
output2 = model2.generate(input_ids,
                        max_length=max_length,
                        num_return_sequences=num_return_sequences,  # Fixed parameter
                        temperature=0.7,
                        top_k=20,  # for more diverse sampling
                        no_repeat_ngram_size=2
                        )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for i, sequence1 in enumerate(output2):
    print(f"Generated sequence after fine tuning {i + 1}: {tokenizer.decode(sequence1)}")
    print("\nEnd of sequence \n")

Generated sequence after fine tuning 1:  python can be very powerful, and it can even be used to kill a python."
"the python is a very large python, weighing in at over 1,000 pounds." he continued, looking at the python.
the black python was a large black snake, with a long tail, a thick body, thick skin, smooth skin and a smooth mouth. it was the most powerful python in the world." i asked, wondering if he could tell me more about the black and white python?

End of sequence 



In [ ]:
from datasets import load_metric


In [ ]:
output2 = model2.generate(input_ids,
                        max_length=max_length,
                        num_return_sequences=num_return_sequences,  # Fixed parameter
                        temperature=0.7,
                        top_k=20,  # for more diverse sampling
                        no_repeat_ngram_size=2
                        )


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
prediction=[tokenizer.decode(sequence1)]
reference=["an image popped into his head glowing just like the grailshaped beacon in that monty python moviea bottle of bourbon "]

rouge=load_metric("rouge")

rouge.compute(predictions=prediction,references=reference)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'rouge1': AggregateScore(low=Score(precision=0.14102564102564102, recall=0.55, fmeasure=0.22448979591836735), mid=Score(precision=0.14102564102564102, recall=0.55, fmeasure=0.22448979591836735), high=Score(precision=0.14102564102564102, recall=0.55, fmeasure=0.22448979591836735)),
 'rouge2': AggregateScore(low=Score(precision=0.07792207792207792, recall=0.3157894736842105, fmeasure=0.12499999999999999), mid=Score(precision=0.07792207792207792, recall=0.3157894736842105, fmeasure=0.12499999999999999), high=Score(precision=0.07792207792207792, recall=0.3157894736842105, fmeasure=0.12499999999999999)),
 'rougeL': AggregateScore(low=Score(precision=0.11538461538461539, recall=0.45, fmeasure=0.1836734693877551), mid=Score(precision=0.11538461538461539, recall=0.45, fmeasure=0.1836734693877551), high=Score(precision=0.11538461538461539, recall=0.45, fmeasure=0.1836734693877551)),
 'rougeLsum': AggregateScore(low=Score(precision=0.14102564102564102, recall=0.55, fmeasure=0.22448979591836735)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ca8349327db443cf7a0e611e2c05329a0bc0a83600c81710848af1edfca83669
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')